In [1]:
"""
3D drift estimation example.

Runs ok on google colab with T4 GPU, but for some reason the CPU multithreading is much slower than on windows
"""

%cd /content
!rm -rf /content/drift-estimation
!git clone https://github.com/qnano/drift-estimation.git
%cd /content/drift-estimation
!cmake .
!make clean & make

/content
Cloning into 'drift-estimation'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 386 (delta 233), reused 284 (delta 145), pack-reused 0
Receiving objects: 100% (386/386), 29.56 MiB | 23.32 MiB/s, done.
Resolving deltas: 100% (233/233), done.
/content/drift-estimation
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for a CUDA compiler
-- Looking for a CUDA compiler - /usr/local/cuda/bin/nvcc
-- The CUDA compiler identification is NVIDIA 11.8.89
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile fea

In [2]:

!nvidia-smi

Wed Oct  4 19:26:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from dme.dme import dme_estimate

# Need to have CUDA >= 10.1 update 2 installed
use_cuda=True


# Simulate an SMLM dataset in 3D with blinking molecules
def smlm_simulation(
        drift_trace,
        fov_width, # field of view size in pixels
        loc_error, # localization error XYZ
        n_sites, # number of locations where molecules blink on and off
        n_frames,
        on_prob = 0.1, # probability of a binding site generating a localization in a frame
        ):

    """
    localization error is set to 20nm XY and 50nm Z precision
    (assumping Z coordinates are in um and XY are in pixels)
    """

    # typical 2D acquisition with small Z range and large XY range
    binding_sites = np.random.uniform([0,0,-1], [fov_width,fov_width,1], size=(n_sites,3))

    localizations = []
    framenum = []

    for i in range(n_frames):
        on = np.random.binomial(1, on_prob, size=n_sites).astype(np.bool)
        locs = binding_sites[on]*1
        # add localization error
        locs += drift_trace[i] + np.random.normal(0, loc_error, size=locs.shape)
        framenum.append(np.ones(len(locs),dtype=np.int32)*i)
        localizations.append(locs)

    return np.concatenate(localizations), np.concatenate(framenum)

n_frames = 10000
fov_width = 200
drift_mean = (0.001,0,0)
drift_stdev = (0.02,0.02,0.02)
loc_error = np.array((0.1,0.1,0.03)) # pixel, pixel, um
# Ground truth drift trace
drift_trace = np.cumsum(np.random.normal(drift_mean, drift_stdev, size=(n_frames,3)), 0)
drift_trace -= drift_trace.mean(0)


localizations, framenum = smlm_simulation(drift_trace, fov_width, loc_error,
                                          n_sites=200,
                                          n_frames=n_frames)
print(f"Total localizations: {len(localizations)}")

crlb = np.ones(localizations.shape) * np.array(loc_error)[None]

estimated_drift,_ = dme_estimate(localizations, framenum,
             crlb,
             framesperbin = 1,  # note that small frames per bin use many more iterations
             imgshape=[fov_width, fov_width],
             coarseFramesPerBin=200,
             coarseSigma=[0.2,0.2,0.2],  # run a coarse drift correction with large Z sigma
             useCuda=use_cuda)

rmsd = np.sqrt(np.mean((estimated_drift-drift_trace)**2, 0))
print(f"RMSD of drift estimate compared to true drift: {rmsd}")

fig,ax=plt.subplots(3, figsize=(7,6))
for i in range(3):
    ax[i].plot(drift_trace[:,i],label='True drift')
    ax[i].plot(estimated_drift[:,i]+0.2,label='Estimated drift')
    ax[i].set_title(['x', 'y', 'z'][i])

    unit = ['px', 'px', 'um'][i]
    ax[i].set_ylabel(f'Drift [{unit}]')
ax[0].legend()
plt.tight_layout()



<ipython-input-3-bdb717f6a435>:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  on = np.random.binomial(1, on_prob, size=n_sites).astype(np.bool)


Total localizations: 200079
Computing XY drift
RCC: Computing image cross correlations. Image stack shape: (10, 410, 410). Size: 6 MB


100%|██████████| 45/45 [00:03<00:00, 11.74it/s]


Computing Z drift
RCC: Computing image cross correlations. Image stack shape: (10, 402, 402). Size: 6 MB


100%|██████████| 45/45 [00:03<00:00, 12.06it/s]

Computing initial coarse drift estimate... (200 frames/bin)



19. Accepting step. Score: 5.575740. Stepsize: 1.597402e-05 [cuda=1, dims=3]: : 20it [01:57, 10.78s/it]